<a href="https://colab.research.google.com/github/koyama-taisei/make-shift-schedule/blob/main/%E3%82%B7%E3%83%95%E3%83%88%E8%87%AA%E5%8B%95%E4%BD%9C%E6%88%90%E3%83%84%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime

# --- グローバル定数 ---
SHIFT_HOURS = {'I': 1.5, 'E': 10.0, 'R': 5.4167, 'T': 3.0, 'Q': 3.5}
STAFF_NAMES_LIST = ["神山", "石塚", "小山", "浅野", "梅津", "福田", "斉藤"]
HOURLY_WAGE = 1782

# Excelファイル構造に関する定数 (DataFrameの0ベースインデックス)
DATE_HEADER_ROW_IDX = 0      # 日付が記載されている行 (Excel物理2行目)
STAFF_NAME_COL_IDX = 0       # スタッフ名が記載されている列 (Excel A列)
STAFF_DATA_START_ROW_IDX = 7 # スタッフ別データ開始行 (Excel物理9行目)
DATE_DATA_START_COL_IDX = 1  # 日付データの開始列 (Excel B列)

# 日次要求シフトの読み取り設定 (Excel物理4行目から5行分)
REQ_SHIFT_START_ROW_IDX = 2  # df.iloc[2] = Excel物理4行目
REQ_SHIFT_NUM_ROWS = 5       # df.iloc[2]からdf.iloc[6]まで (物理4～8行目)

# --- ヘルパー関数 ---
def get_excel_col_name(col_idx):
    """0から始まる列インデックスをExcelの列名（A, B, ..., Z, AA, AB, ...）に変換する"""
    if col_idx < 0:
        return "N/A"
    string = ""
    temp_col_idx = col_idx
    while temp_col_idx >= 0:
        string = chr(ord('A') + temp_col_idx % 26) + string
        temp_col_idx = temp_col_idx // 26 - 1
    return string

def print_excel_cell_details(df_loaded, max_rows_to_print=20, max_cols_to_print=35): # 引数名を修正
    """
    読み込まれたDataFrameの内容を、Excelのセル番地風に表示する。
    df_loadedは header=0 (またはpandasのデフォルト) で読み込まれたものを想定。
    """
    if df_loaded is None:
        print("--- (DataFrameがNoneのため、セル内容は表示できません) ---")
        return

    print(f"\n--- DataFrameの内容確認 (Excel1行目はヘッダーとして読み込まれています) ---")
    print(f"--- 表示は最大 {max_rows_to_print} データ行 × {max_cols_to_print} 列に制限 ---")

    actual_cols_to_print = min(df_loaded.shape[1], max_cols_to_print)
    actual_rows_to_print = min(df_loaded.shape[0], max_rows_to_print)

    print("\nExcel物理1行目 (DataFrame列ヘッダー):")
    header_info = []
    for c_idx in range(actual_cols_to_print):
        excel_col_letter = get_excel_col_name(c_idx)
        try:
            header_val = df_loaded.columns[c_idx]
            header_info.append(f"  セル {excel_col_letter}1 (列名: {c_idx}): '{header_val}' (型: {type(header_val).__name__})")
        except IndexError:
            header_info.append(f"  セル {excel_col_letter}1 (列名: {c_idx}): 範囲外")

    print("\n".join(header_info))
    if df_loaded.shape[1] > max_cols_to_print:
        print(f"  ... さらに {df_loaded.shape[1] - max_cols_to_print} 列のヘッダーがありますが、表示を省略 ...")

    print("\nExcel物理2行目以降 (DataFrameデータ行):")
    for r_idx in range(actual_rows_to_print):
        excel_row_num = r_idx + 2
        print(f"\n  Excel物理{excel_row_num}行目 (df.iloc[{r_idx}]):")
        row_info = []
        for c_idx in range(actual_cols_to_print):
            excel_col_letter = get_excel_col_name(c_idx)
            cell_ref = f"{excel_col_letter}{excel_row_num}"
            raw_value = df_loaded.iloc[r_idx, c_idx]
            value_type = type(raw_value).__name__
            processed_value_str = ""; cell_display_value = f"'{raw_value}'"
            if pd.isna(raw_value): cell_display_value = "<空白/NA>"
            if isinstance(raw_value, str): processed_value_str = f", 処理後文字列: '{raw_value.strip().upper()}'"
            if pd.notna(raw_value): row_info.append(f"    セル {cell_ref}: 生の値={cell_display_value} (型: {value_type}){processed_value_str}")

        if row_info: print("\n".join(row_info))
        elif actual_cols_to_print > 0 : print(f"    (この行の表示対象列は全て空白/NAです)")
        if df_loaded.shape[1] > max_cols_to_print and row_info : print(f"    ... さらに {df_loaded.shape[1] - max_cols_to_print} 列ありますが、表示を省略 ...")

    if df_loaded.shape[0] > max_rows_to_print: print(f"\n... さらに {df_loaded.shape[0] - max_rows_to_print} データ行がありますが、表示を省略しました ...")
    print("--- DataFrameの内容確認ここまで ---")

def load_excel_data(file_path):
    print(f"--- ファイル '{file_path}' の読み込みを開始します ---")
    try:
        df = pd.read_excel(file_path, engine='openpyxl', header=0)
        print("--- ファイルの読み込みに成功しました ---")
        print(f"--- DataFrameの形状 (行数, 列数): {df.shape} ---")
        print(f"--- DataFrameの列名 (最初の10件): {df.columns.tolist()[:10]} ---")
        return df
    except FileNotFoundError: print(f"エラー: ファイル '{file_path}' が見つかりませんでした。"); return None
    except Exception as e: print(f"Excelファイルの読み込み中にエラーが発生しました: {e}"); return None

def identify_staff_info(df, config_staff_names_list):
    staff_row_indices_map = {}
    print(f"--- スタッフ情報読み取り開始 (Excel A列、物理{STAFF_DATA_START_ROW_IDX+1}行目から) ---")
    for idx in range(STAFF_DATA_START_ROW_IDX, len(df.index)):
        staff_name_candidate = str(df.iloc[idx, STAFF_NAME_COL_IDX]).strip()
        if staff_name_candidate in config_staff_names_list:
            if staff_name_candidate not in staff_row_indices_map: staff_row_indices_map[staff_name_candidate] = idx
        elif pd.notna(df.iloc[idx, STAFF_NAME_COL_IDX]) and staff_name_candidate != "nan" and staff_name_candidate:
             print(f"情報: ファイルのスタッフ名 '{staff_name_candidate}' (df行 {idx}) は、定義済みリストにないため処理対象外。")
    active_staff_list = [name for name in config_staff_names_list if name in staff_row_indices_map]
    if not active_staff_list: print("エラー: 有効なスタッフが一人も見つかりませんでした。"); return None, None
    print(f"--- 処理対象スタッフ: {active_staff_list} ---")
    return staff_row_indices_map, active_staff_list

def identify_date_columns_from_df(df):
    if DATE_HEADER_ROW_IDX >= len(df.index):
        print(f"エラー: DataFrameの行数が不足。日付ヘッダー行(idx {DATE_HEADER_ROW_IDX})を読み取れません。"); return None, None
    date_col_names_list, date_col_day_strings_list = [], []
    print(f"--- 日付列の特定を開始 (Excel物理2行目(df idx {DATE_HEADER_ROW_IDX})、B列(df col idx {DATE_DATA_START_COL_IDX})から) ---")
    for col_idx in range(DATE_DATA_START_COL_IDX, df.shape[1]):
        current_col_name = df.columns[col_idx]
        val_in_date_header_row = df.iloc[DATE_HEADER_ROW_IDX, col_idx]
        day_num_str = None
        try:
            if pd.isna(val_in_date_header_row): continue
            if isinstance(val_in_date_header_row, (datetime.datetime, pd.Timestamp)): day_num_str = str(val_in_date_header_row.day)
            elif isinstance(val_in_date_header_row, str):
                cleaned_str = val_in_date_header_row.strip(); parts = cleaned_str.split('/')
                if '/' in cleaned_str and len(parts) >= 2: day_val = int(parts[1].strip()); day_num_str = str(day_val) if 1 <= day_val <= 31 else None
                else: day_val = int(cleaned_str); day_num_str = str(day_val) if 1 <= day_val <= 31 else None
            elif isinstance(val_in_date_header_row, (int, float)): day_val = int(val_in_date_header_row); day_num_str = str(day_val) if 1 <= day_val <= 31 else None
        except: continue
        if day_num_str: date_col_names_list.append(current_col_name); date_col_day_strings_list.append(day_num_str)
    print(f"--- 日付列の特定処理終了。特定数: {len(date_col_day_strings_list)} ---")
    if date_col_day_strings_list: print(f"--- 特定日付 (最初の5件): {date_col_day_strings_list[:5]} ---")
    if not date_col_day_strings_list: print("エラー: 有効な日付列が特定できませんでした。"); return None, None
    return date_col_names_list, date_col_day_strings_list

def initialize_schedule_and_counts(active_staff_names, date_cols_day_strings):
    output_df = pd.DataFrame(index=active_staff_names, columns=date_cols_day_strings).fillna('')
    staff_total_hours_map = {name: 0.0 for name in active_staff_names}
    staff_i_counts_map = {name: 0 for name in active_staff_names if name != "神山"}
    staff_e_counts_map = {name: 0 for name in active_staff_names if name != "神山"}
    staff_unavailability_map = {name: {day_s: set() for day_s in date_cols_day_strings} for name in active_staff_names}
    return output_df, staff_total_hours_map, staff_i_counts_map, staff_e_counts_map, staff_unavailability_map

def load_staff_preassignments(df, active_staff_names, staff_row_indices,
                              date_cols_names, date_cols_day_strings,
                              output_schedule_df, staff_total_hours,
                              staff_i_shift_counts, staff_e_shift_counts,
                              staff_unavailability):
    print("\n--- 事前割り当てと「×」および個別不可シフトの読み込みを開始 ---")
    for staff_name in active_staff_names:
        staff_df_actual_row_idx = staff_row_indices[staff_name]
        for i, day_str in enumerate(date_cols_day_strings):
            original_col_name = date_cols_names[i]
            try: col_loc_for_staff_data = df.columns.get_loc(original_col_name)
            except KeyError: print(f"警告: 事前割り当て列名 '{original_col_name}' (日:{day_str}) がdfにありません。"); continue
            original_value = df.iloc[staff_df_actual_row_idx, col_loc_for_staff_data]
            current_val = str(original_value).strip().upper() if pd.notna(original_value) else ''
            if current_val == '×': output_schedule_df.loc[staff_name, day_str] = '×'
            elif current_val == 'RT×': staff_unavailability[staff_name][day_str].update(['R', 'T'])
            elif current_val == 'I×': staff_unavailability[staff_name][day_str].add('I')
            elif current_val == 'R×': staff_unavailability[staff_name][day_str].add('R')
            elif current_val in SHIFT_HOURS:
                 output_schedule_df.loc[staff_name, day_str] = current_val
                 staff_total_hours[staff_name] += SHIFT_HOURS[current_val]
                 if current_val == 'I' and staff_name != "神山": staff_i_shift_counts[staff_name] += 1
                 elif current_val == 'E' and staff_name != "神山": staff_e_shift_counts[staff_name] += 1
                 print(f"情報: 日 {day_str}、スタッフ {staff_name} に事前割り当てシフト '{current_val}' を確認。累計時間: {staff_total_hours[staff_name]:.2f}")
            elif current_val and current_val != "NAN":
                 print(f"  [事前割当診断] スタッフ: {staff_name}, 日: {day_str} (列 {original_col_name}), 不明な値: '{current_val}'。スキップ。")
    print("--- 初期総勤務時間計算完了 ---")

def get_daily_shift_requirements_list(df, day_str, original_col_name):
    try: col_loc = df.columns.get_loc(original_col_name)
    except KeyError: print(f"警告: 日 {day_str} の列名 '{original_col_name}' がdfにありません。要求シフトは空。"); return []
    explicit_shifts = set(); is_e_day = False
    for r_offset in range(REQ_SHIFT_NUM_ROWS):
        check_row = REQ_SHIFT_START_ROW_IDX + r_offset
        if check_row >= len(df.index): break
        code_val = df.iloc[check_row, col_loc]
        code = str(code_val).strip().upper() if pd.notna(code_val) else ''
        if not code: continue
        if code == 'E' and code in SHIFT_HOURS: explicit_shifts = {'E'}; is_e_day = True; break
        elif code in SHIFT_HOURS: explicit_shifts.add(code)
    demand = []
    if is_e_day: demand = ['E']
    else:
        r_is = 'R' in explicit_shifts; t_is = 'T' in explicit_shifts
        if not r_is and not t_is:
            demand = ['Q', 'Q']
            if 'I' in explicit_shifts: demand.insert(0, 'I')
        else:
            if 'I' in explicit_shifts: demand.append('I')
            if 'R' in explicit_shifts: demand.append('R')
            if 'T' in explicit_shifts: demand.append('T')
            if 'Q' in explicit_shifts: demand.extend(['Q', 'Q'])
    if demand: print(f"情報: 日 {day_str} (列 {original_col_name})、施設による初期要求シフト: {demand}")
    else: print(f"情報: 日 {day_str} (列 {original_col_name}) 有効な要求シフトなし。")
    return demand

def assign_shifts_for_one_day(day_str, initial_demand_list, active_staff_names,
                              output_schedule_df, staff_total_hours, staff_i_shift_counts,
                              staff_e_shift_counts, staff_unavailability):
    if not initial_demand_list: return
    temp_demand = list(initial_demand_list)
    for s_check in active_staff_names:
        pre_assigned = output_schedule_df.loc[s_check, day_str]
        if pre_assigned and pre_assigned != '×':
            for pa_s in pre_assigned.split(','):
                if pa_s in temp_demand: temp_demand.remove(pa_s)
    shifts_to_auto = temp_demand
    if not shifts_to_auto: print(f"情報: 日 {day_str}、全ての要求は事前割り当て等で充足済み。"); return
    print(f"情報: 日 {day_str}、自動割り当て対象シフト: {shifts_to_auto}")
    for shift_to_assign in shifts_to_auto:
        candidates = []
        for s_name in active_staff_names:
            if output_schedule_df.loc[s_name, day_str] == '×': continue
            if s_name == "神山" and shift_to_assign in ['R', 'T', 'Q', 'E']: continue
            specific_unavail = staff_unavailability[s_name].get(day_str, set())
            if shift_to_assign in specific_unavail: continue
            current_shifts = output_schedule_df.loc[s_name, day_str]; can_add = False
            if not current_shifts: can_add = True
            else:
                if ',' not in current_shifts:
                    if shift_to_assign == 'I' and current_shifts in ['R', 'T', 'Q']: can_add = True
                    elif shift_to_assign in ['R', 'T', 'Q'] and current_shifts == 'I': can_add = True
            if can_add: candidates.append(s_name)
        if not candidates: print(f"警告: 日 {day_str}, シフト {shift_to_assign}: 利用可能なスタッフがいません。"); continue
        chosen = None
        if shift_to_assign == 'E':
            nk_cand = [s for s in candidates if s != "神山"]
            if not nk_cand: print(f"情報: 日 {day_str}, シフト E: 神山さん以外の候補がいません。"); continue
            g0e = sorted([s for s in nk_cand if staff_e_shift_counts.get(s,0) == 0], key=lambda srt: staff_total_hours[srt])
            gXe = sorted([s for s in nk_cand if staff_e_shift_counts.get(s,0) > 0], key=lambda srt: staff_total_hours[srt])
            if g0e: chosen = g0e[0]
            elif gXe: chosen = gXe[0]
        elif shift_to_assign == 'I':
            k_c = None; o_c = []
            for sc_i in candidates:
                if sc_i == "神山": k_c = sc_i
                else: o_c.append(sc_i)
            if o_c: o_c.sort(key=lambda srt: (staff_i_shift_counts.get(srt,0), staff_total_hours[srt])); chosen = o_c[0]
            elif k_c: chosen = k_c
        else: candidates.sort(key=lambda srt: staff_total_hours[srt]); chosen = candidates[0] if candidates else None
        if not chosen: print(f"警告: 日 {day_str}, シフト {shift_to_assign}: 適切なスタッフが見つかりませんでした。"); continue
        current_cell_val = output_schedule_df.loc[chosen, day_str]; new_text_val = shift_to_assign
        if current_cell_val and current_cell_val != '×':
            combined_s_set = set(current_cell_val.split(',')); combined_s_set.add(shift_to_assign)
            new_text_val = ",".join(sorted(list(combined_s_set)))
        output_schedule_df.loc[chosen, day_str] = new_text_val
        staff_total_hours[chosen] += SHIFT_HOURS[shift_to_assign]
        if shift_to_assign == 'I' and chosen != "神山": staff_i_shift_counts[chosen] += 1
        elif shift_to_assign == 'E' and chosen != "神山": staff_e_shift_counts[chosen] += 1
        print(f"  -> 日 {day_str}: スタッフ {chosen} にシフト {shift_to_assign} を追加 (セル内容: '{new_text_val}')。累計時間: {staff_total_hours[chosen]:.2f}")

def print_final_summary_info(active_staff_names, staff_total_hours, staff_salaries,
                             staff_e_shift_counts, staff_i_shift_counts):
    print("\n--- 最終総勤務時間: ---")
    for sn in active_staff_names: print(f"- {sn}: {staff_total_hours.get(sn, 0.0):.2f} 時間")
    print("\n--- 推定給与 (円): ---")
    for sn in active_staff_names: print(f"- {sn}: {staff_salaries.get(sn, 0.0):,.0f} 円")
    print("\n--- Eシフト割り当て回数 (神山以外): ---")
    all_e_req_met = True; non_k_exists = any(name != "神山" for name in active_staff_names)
    for sn in active_staff_names:
        if sn == "神山": continue
        c = staff_e_shift_counts.get(sn, 0); print(f"- {sn}: {c} 回")
        if c == 0: all_e_req_met = False; print(f"  警告: {sn} さんはEシフトが0回です。")
    if all_e_req_met and non_k_exists : print("情報: 神山さん以外のスタッフにEシフト割り当て試行完了。")
    elif not non_k_exists: print("情報: 神山さん以外のスタッフがいません。")
    print("\n--- Iシフト割り当て回数 (神山以外): ---")
    for sn in active_staff_names:
        if sn == "神山": continue
        print(f"- {sn}: {staff_i_shift_counts.get(sn, 0)} 回")

def save_to_excel(final_df, output_filename="generated_schedule.xlsx"):
    try:
        df_xl = final_df.reset_index(); df_xl.rename(columns={'index': 'スタッフ名'}, inplace=True)
        date_cols = [col for col in final_df.columns if col not in ["総勤務時間", "推定給与(円)"]]
        ordered_cols = ['スタッフ名'] + date_cols + \
                       [col for col in ["総勤務時間", "推定給与(円)"] if col in df_xl.columns]
        final_ordered_cols = [col for col in ordered_cols if col in df_xl.columns]
        df_xl[final_ordered_cols].to_excel(output_filename, index=False)
        print(f"\nシフト表の生成に成功し、'{output_filename}' に保存しました。")
    except Exception as e: print(f"\nExcelファイルへの保存中にエラー: {e}")

# =============================================
# メインのシフト作成統括関数
# =============================================
def generate_shift_schedule_main_with_diagnostics(file_path="2025シフト.xlsx"):
    df = load_excel_data(file_path)
    if df is None: return None

    print_excel_cell_details(df, max_rows_to_print=15, max_cols_to_print=35) # ★★★ 修正箇所 ★★★

    print(f"\n--- 時給: {HOURLY_WAGE} 円 ---")

    staff_row_indices, active_staff_names = identify_staff_info(df, STAFF_NAMES_LIST)
    if not active_staff_names: return None

    date_cols_names, date_cols_day_strings = identify_date_columns_from_df(df)
    if not date_cols_day_strings: return None

    output_schedule_df, staff_total_hours, \
    staff_i_shift_counts, staff_e_shift_counts, \
    staff_unavailability = initialize_schedule_and_counts(active_staff_names, date_cols_day_strings)

    load_staff_preassignments(df, active_staff_names, staff_row_indices,
                              date_cols_names, date_cols_day_strings,
                              output_schedule_df, staff_total_hours,
                              staff_i_shift_counts, staff_e_shift_counts,
                              staff_unavailability)

    print("\n--- 日次要求シフト読み取りと割り当て処理を開始します ---")
    current_req_shift_start_row = REQ_SHIFT_START_ROW_IDX
    current_req_shift_num_rows = REQ_SHIFT_NUM_ROWS
    print(f"--- 日次シフトタイプ読み取り範囲: DataFrameインデックス {current_req_shift_start_row} から {current_req_shift_start_row + current_req_shift_num_rows - 1} ---")
    if current_req_shift_start_row + current_req_shift_num_rows > len(df.index) :
         print(f"警告: シフトタイプ読み取り範囲がDataFrameの行数を超えています。実際の読み取り行数を調整します。")
         current_req_shift_num_rows = len(df.index) - current_req_shift_start_row
         print(f"調整後の日次シフトタイプ読み取り行数: {current_req_shift_num_rows}")

    for i, day_str in enumerate(date_cols_day_strings):
        original_col_name = date_cols_names[i]
        initial_demand = get_daily_shift_requirements_list(df, day_str, original_col_name)
        assign_shifts_for_one_day(day_str, initial_demand, active_staff_names,
                                  output_schedule_df, staff_total_hours, staff_i_shift_counts,
                                  staff_e_shift_counts, staff_unavailability)

    staff_salaries = {name: hours * HOURLY_WAGE for name, hours in staff_total_hours.items()}
    print_final_summary_info(active_staff_names, staff_total_hours, staff_salaries,
                             staff_e_shift_counts, staff_i_shift_counts)
    final_output_df = output_schedule_df.copy()
    final_output_df["総勤務時間"] = pd.Series(staff_total_hours)
    final_output_df["推定給与(円)"] = pd.Series(staff_salaries)
    return final_output_df

# --- メイン処理の実行 ---
if __name__ == "__main__":
    schedule_df = generate_shift_schedule_main_with_diagnostics()
    if schedule_df is not None:
        save_to_excel(schedule_df)
    else:
        print("\nシフト表の生成に失敗しました。")

--- ファイル '2025シフト.xlsx' の読み込みを開始します ---
--- ファイルの読み込みに成功しました ---
--- DataFrameの形状 (行数, 列数): (14, 32) ---
--- DataFrameの列名 (最初の10件): [2025, 5, 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'] ---

--- DataFrameの内容確認 (Excel1行目はヘッダーとして読み込まれています) ---
--- 表示は最大 15 データ行 × 35 列に制限 ---

Excel物理1行目 (DataFrame列ヘッダー):
  セル A1 (列名: 0): '2025' (型: int)
  セル B1 (列名: 1): '5' (型: int)
  セル C1 (列名: 2): 'Unnamed: 2' (型: str)
  セル D1 (列名: 3): 'Unnamed: 3' (型: str)
  セル E1 (列名: 4): 'Unnamed: 4' (型: str)
  セル F1 (列名: 5): 'Unnamed: 5' (型: str)
  セル G1 (列名: 6): 'Unnamed: 6' (型: str)
  セル H1 (列名: 7): 'Unnamed: 7' (型: str)
  セル I1 (列名: 8): 'Unnamed: 8' (型: str)
  セル J1 (列名: 9): 'Unnamed: 9' (型: str)
  セル K1 (列名: 10): 'Unnamed: 10' (型: str)
  セル L1 (列名: 11): 'Unnamed: 11' (型: str)
  セル M1 (列名: 12): 'Unnamed: 12' (型: str)
  セル N1 (列名: 13): 'Unnamed: 13' (型: str)
  セル O1 (列名: 14): 'Unnamed: 14' (型: str)
  セル P1 (列名: 15): 'Unnamed: 15' (型: str)
  セル Q1